In [1]:
import tweepy
import time
import json
import speech_recognition as sr
recognizer = sr.Recognizer()
import pyttsx3 

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJVAyAEAAAAACRa1kXaV8Yh3odafpbp0gaoLvCs%3DxVv3I5pya0j8ukWDU3bMb6QT1CxTVeGMZhfomCu0y0pKvf6MuT'
consumer_key = 'MFM5QnBsdzFXa1hzb3AzeWgtR3I6MTpjaQ'
consumer_secret = 'c3Z6ZPz8FiW3qEPiqjQ_Igam_7fV_UnLI8xiZ6A_NqrwGPfxSq'
access_token = '1877758635545415680-Xk7lBzQKn5NmvOSSQdIkhHAZRAcaC3'
access_token_secret = '21o3vWrnd3H2YYhe9eLKqRoY8rkSgWQesfA7ynoLdqEyU'
# Crear el cliente de la API
#auth = tweepy.OAuth1UserHandler(
 #   consumer_key, consumer_secret, access_token, access_token_secret
#)
#client = tweepy.API(auth)

client = tweepy.Client(bearer_token=bearer_token)

In [2]:
def listen_command():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        speak("Escuchando comando...")
        try:
            audio1 = recognizer.listen(source, timeout=20, phrase_time_limit=10)
            command = recognizer.recognize_google(audio1)
            print("Comando detectado:", command)
            return command
        except sr.WaitTimeoutError:
            speak("No se detectó ningún comando.")
        except sr.UnknownValueError:
            speak("No se pudo entender el comando.")
        except sr.RequestError:
            speak("Error en la solicitud de la API de Google.")
    return None

In [3]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    print(text)
    engine.runAndWait()

In [5]:
def buscar_tweets(termino, wait_time=30, output_file="resultados.json"):
    data = []
    while True:
        try:
            response = client.search_recent_tweets(query=termino, max_results=10, expansions="author_id,in_reply_to_user_id,referenced_tweets.id")
            for tweet in response.data:
                author_id = tweet.author_id

                try:
                    user = client.get_user(ids=author_id, user_fields="verified")
                    is_author_verified = user.data[0].verified
                except tweepy.errors.BadRequest as e:
                    if "No user matches" in str(e):
                        print(f"Author ID {author_id} not found. Attempting username lookup...")
                        try:
                            user = client.get_user(username=tweet.author_username, user_fields="verified")
                            is_author_verified = user.data[0].verified
                        except tweepy.errors.BadRequest as e:
                            if "No user matches" in str(e):
                                print(f"Username {tweet.author_username} not found. Skipping tweet.")
                                continue
                            else:
                                raise

                tweet_data = {
                    "author_id": author_id,
                    "is_author_verified": is_author_verified,
                    "text": tweet.text,
                    "mentions": []
                }

                if 'entities' in tweet and 'mentions' in tweet['entities']:
                    for mention in tweet['entities']['mentions']:
                        user_id = mention['id']
                        try:
                            user = client.get_user(ids=user_id, user_fields="verified")
                            mention_data = {
                                'user_id': user_id,
                                'is_verified': user.data[0].verified,
                                'username': user.data[0].username
                            }
                        except tweepy.errors.BadRequest as e:
                            if "No user matches" in str(e):
                                print(f"Mencion ID {user_id} no encontrado.")
                                continue 
                            else:
                                raise
                        tweet_data['mentions'].append(mention_data)

                data.append(tweet_data)

            break
        except Exception as e:
            if "429" in str(e):
                print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Otro error ocurrió: {e}")
                break

    # Guardar los datos en un archivo JSON
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

In [5]:

if __name__ == "__main__":
    recognizer = sr.Recognizer()
    while True:
        termino = listen_command()
        buscar_tweets3(termino)
        speak(f"Escuché {termino}. ¿Quieres analizar esta empresa o negocio?")
        with sr.Microphone() as source:
            print("Escuchando confirmación...")
            try:
                # Primer intento de captura de comando
                audio2 = recognizer.listen(source, timeout=20, phrase_time_limit=10)
                command = recognizer.recognize_google(audio2)
                if command:
                    if "sí" in command.lower():  # Agrega la condición para detener la búsqueda si es 'sí'
                        speak(f"Buscando tweets sobre {termino}.")
                        buscar_tweets3(termino)
                    elif "no" in command.lower():
                        speak("Volvere a preguntar.")
            except sr.WaitTimeoutError:
                speak("No se detectó la confirmación.")
            except sr.UnknownValueError:
                speak("No se pudo entender la confirmación.")
            except sr.RequestError:
                speak("Error en la solicitud de la API de Google.")
return speak("Fin")

Escuchando comando...
Comando detectado: Coca-Cola
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicit

KeyboardInterrupt: 

In [ ]:
def buscar_tweets2(termino, wait_time=30):
    while True:
        try:
            response = client.search_recent_tweets(query=termino, max_results=10)
            for tweet in response.data:
                print(f"Autor: {tweet.author_id}: Tweet: {tweet.text}")
            break
        except Exception as e:
            if "429" in str(e):
                print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Otro error ocurrió: {e}")
                break

In [4]:
def buscar_tweets3(termino, wait_time=30, output_file="resultados.json"):
    data = []
    while True:
        try:
            response = client.search_recent_tweets(query=termino, max_results=10, expansions="author_id")
            for tweet in response.data:
                author_id = tweet.author_id

                # Obtener información del autor
                user = client.get_user(ids=author_id, user_fields="verified")
                is_author_verified = user.data[0].verified

                tweet_data = {
                    "author_id": author_id,
                    "is_author_verified": is_author_verified,
                    "text": tweet.text
                }

                data.append(tweet_data)

            break
        except Exception as e:
            if "429" in str(e):
                print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Otro error ocurrió: {e}")
                break

    # Guardar los datos en un archivo JSON
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

In [3]:

def buscar_cuentas_verificadas(termino, wait_time=15):
    try:
        response = client.search_recent_tweets(
            query=termino,
            expansions=["author_id"],  # Expandir para obtener detalles del autor
            tweet_fields=["created_at"],  # Obtener campos adicionales de los tweets
            user_fields=["verified", "username", "name"],  # Campos adicionales de los usuarios
            max_results=10  # Número de resultados a obtener
        )
        
        if not response.data:
            print("No se encontraron tweets para el término proporcionado.")
            return

        usuarios = {user["id"]: user for user in response.includes["users"]}

        cuentas_verificadas = [
            usuarios[tweet.author_id]
            for tweet in response.data
            if usuarios[tweet.author_id]["verified"]
        ]

        if cuentas_verificadas:
            print(f"Cuentas verificadas relacionadas con el término '{termino}':\n")
            for cuenta in cuentas_verificadas:
                print(f"Nombre: {cuenta['name']}")
                print(f"Usuario: @{cuenta['username']}")
                print(f"ID: {cuenta['id']}")
                print("-" * 40)
        else:
            print(f"No se encontraron cuentas verificadas para el término '{termino}'.")

    except Exception as e:
        if "429" in str(e):
            print(f"Demasiadas solicitudes. Esperando {wait_time} segundos")
            time.sleep(wait_time)
            buscar_cuentas_verificadas(termino, wait_time)
        else:
            print(f"Error al buscar cuentas verificadas: {e}")

In [ ]:
def cuenta_verificada(username, wait_time = 30):

    try: 
        user = client.get_user(username=username, user_fields = ['id', 'username', 'name', 'created_at', 'location', 'description', 'public_metrics', 'verified'])
        
        user_data = user.data
        
        print(f"Nombre: {user_data.name}")
        print(f"Usuario: {user_data.username}")
        print(f"ID: {user_data.id}")
        print(f"Fecha de Creacion: {user_data.created_at}")
        print(f"Descripcion: {user_data.description}")
        print(f"Locacion: {user_data.location}")
        print(f"Seguidores: {user_data.public_metrics['followers_count'}}")
        print(f"Seguidos: {user_data.public_metrics['following_count'}}")
        print(f"Tweets realizados: {user_data.public_metrics['tweets_count'}}")
        
        if user_data.verified:
            print(f"{user_data.name} es una cuenta verificada")
        else:
            print(f"{user_data.name} NO es una cuenta verificada")
        
        return json.dumps(user_data, indent = 4)
        
    except tweepy.errors.TooManyRequests:
        print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
        time.sleep(wait_time)
        return cuenta_verificada(username, wait_time=wait_time)
    except tweepy.errors.NotFound:
        print("La cuenta no existe.")
    except tweepy.TweepyException as e:
        print(f"Error al recuperar información: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")


        
    


In [ ]:
def buscar_usuarios(query, wait_time=30):
    try:
        users = client.search_users(query=q)
        for user in users.data:
            print(user.username)
            
    except tweepy.errors.TooManyRequests:
        print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
        time.sleep(wait_time)
        return cuenta_verificada(username, wait_time=wait_time)
    except tweepy.TweepyException as e:
        print(f"Error al recuperar información: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
